#### ​Initialize, Authenticate, and Configure Remote Execution Environment

##### Start a Cybershuttle Remote Executor: This will submit a remote job to Anvil
- **Cluster** - Anvil & Jetstream
- **Community Allocation** - Gkeyll
- **Requested Resources** 
    - 4 CPUs, 4GB Memory, 60 Minutes, Shared Queue
    - 1 CPUs, 60 Minutes, Cloud Queue

##### This remote job enables the generation of plotE_z with post gkeyll code.


In [ ]:
%pip install -qU "airavata-python-sdk[notebook]"
import airavata_jupyter_magic

%authenticate

# PLEASE NOTE: At a given time, ONLY run a single job in one of the available clusters: Anvil OR Jetstream. 
# Anvil is the default cluster and if you need to run on Jetstream; comment the Anvil and uncomment the Jetstream
# Anvil
%request_runtime test_cpu --file=cybershuttle.yml --group=Gkeyll --walltime=60 --use=AnvilCPU:shared
# Jetstream
#%request_runtime test_cpu --file=cybershuttle.yml --group=Gkeyll --walltime=60 --use=Gkeyll:cloud

%wait_for_runtime test_cpu --live
%switch_runtime test_cpu

In [ ]:
#Defininf the data paths in remote clusters
%ls -l /data

#Anvil and Jetstream data path
data_path = "/data/gemReconnection"

!ls /data/gemReconnection

#Note: This cell may execute longer if the job is not yet active in the remote cluster. Please pause till the files are listed below

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import postgkyl as pg

run = Path.cwd()

def getModelType():
    frame = 0
    models = ["5m", "10m"]
    for model in models:
        path = Path(f"{data_path}/rt_{model}_gem_gzero-field_{frame}.gkyl")
        if path.is_file():
            return model
    error = "Failed to find input file " + str(path)
    assert False, error

In [ ]:
frame = 0
model = getModelType()
filename = run / f"{data_path}/rt_{model}_gem_gzero-field_{frame}.gkyl"
filename = str(filename)
print(filename)

In [ ]:
gdata = pg.GData(filename)

vals = gdata.get_values() # cell-center values, shape is Ny * Nx * Ncomponents
grid = gdata.get_grid() # cell corner coordinates
ndim = gdata.get_num_dims() # number of spatial dimensions

assert len(vals.shape) == ndim+1 and len(grid) == ndim

icomp = 3
compName = r'$E_z^2$'
fileName = "E_z.png"

my_vals = vals[..., icomp]
x, y = grid

fig, ax = plt.subplots()

im = ax.pcolormesh(x, y, my_vals.T)
fig.colorbar(im, ax=ax)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title(compName)
ax.set_aspect(1)

plt.savefig(fileName)

#### To Stop the Agent Run

In [ ]:
%stop_runtime test_cpu
%switch_runtime local